# Running Native Functions


Two of the previous notebooks showed how to [execute semantic functions inline](./03-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

- Writing logic around how to run a prompt that changes the prompt's outcome.
- Using external data sources to gather data to concatenate into your prompt.
- Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the SK Python framework.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


In [ ]:
!python -m pip install semantic-kernel==0.5.1.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_chat_service = AzureChatCompletion(
        deployment_name="turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_chat_service("chat_completion", azure_chat_service)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)
    kernel.add_chat_service("chat-gpt", oai_chat_service)

Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.


First, let's create our native function.


In [ ]:
import random
from semantic_kernel.plugin_definition import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between 3-x.
    """

    @kernel_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Corgis on an adventure. `$input` is a default variable semantic functions can use.


In [ ]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long
"""

corgi_story = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    description="Write a short story about two Corgis on an adventure",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

generate_number_plugin = kernel.import_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")

In [ ]:
# Run the number generator
generate_number_three_or_higher = generate_number_plugin["GenerateNumberThreeOrHigher"]
number_result = await generate_number_three_or_higher(6)
print(number_result)

In [ ]:
story = await corgi_story.invoke(input=number_result.result)

In [ ]:
print("Generating a corgi story exactly {} paragraphs long: ".format(number_result.result))
print("=====================================================")
print(story)

## Context Variables

That works! But let's expand on our example to make it more generic.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the `semantic_kernel.ContextVariables` class to do hold these variables.


In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_chat_service = AzureChatCompletion(
        deployment_name="turbo", endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_chat_service("chat_completion", azure_chat_service)
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_chat_service = OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id)
    kernel.add_chat_service("chat-gpt", oai_chat_service)

Let's start with the native function. Notice that we're also adding `@kernel_function_context_parameter` decorators to the function here to provide context about what variables need to be provided to the function, and any defaults for those inputs. Using the `@kernel_function_context_parameter` decorator provides the name, description and default values for a function's inputs to the [planner.](./05-using-the-planner.ipynb)


In [ ]:
import random
from semantic_kernel.plugin_definition import kernel_function, kernel_function_context_parameter
from semantic_kernel import KernelContext


class GenerateNumberPlugin:
    """
    Description: Generate a number between a min and a max.
    """

    @kernel_function(
        description="Generate a random number between min and max",
        name="GenerateNumber",
    )
    @kernel_function_context_parameter(name="min", description="Minimum number of paragraphs.")
    @kernel_function_context_parameter(name="max", description="Maximum number of paragraphs.", default_value="10")
    def generate_number(self, context: KernelContext) -> str:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(int(context["min"]), int(context["max"])))
        except ValueError as e:
            print(f"Invalid input {context['min']} {context['max']}")
            raise e

In [ ]:
generate_number_plugin = kernel.import_plugin(GenerateNumberPlugin(), "GenerateNumberPlugin")
generate_number = generate_number_plugin["GenerateNumber"]

Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our CorgiStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.


In [ ]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

corgi_story = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    description="Write a short story about two Corgis on an adventure",
    max_tokens=500,
    temperature=0.5,
    top_p=0.5,
)

Now we can call this using our `invoke` function by passing in our `context_variables` in the `variables` parameter.


In [ ]:
context_variables = sk.ContextVariables(variables={"min": "1", "max": "5", "language": "Spanish"})

Let's add a paragraph count to our context variables


In [ ]:
num = await generate_number.invoke(variables=context_variables)
context_variables["paragraph_count"] = num.result

In [ ]:
# Pass the output to the semantic story function
story = await corgi_story.invoke(variables=context_variables)

In [ ]:
print(
    "Generating a corgi story exactly {} paragraphs long in {} language: ".format(
        context_variables["paragraph_count"], context_variables["language"]
    )
)
print("=====================================================")
print(story)

## Calling Native Functions within a Semantic Function

One neat thing about the Semantic Kernel is that you can also call native functions from within Semantic Functions!

We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}`. You can read more about our prompte templating syntax [here](../../../docs/PROMPT_TEMPLATE_LANGUAGE.md).


In [ ]:
import random
from semantic_kernel.plugin_definition import kernel_function


class GenerateNamesPlugin:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @kernel_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @kernel_function(description="Generate character names", name="generate_names")
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {"Hoagie", "Hamilton", "Bacon", "Pizza", "Boots", "Shorts", "Tuna"}
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [ ]:
generate_names_plugin = kernel.import_plugin(GenerateNamesPlugin(), plugin_name="GenerateNames")
generate_names = generate_names_plugin["generate_names"]

In [ ]:
sk_prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The two names of the corgis are {{GenerateNames.generate_names}}
"""

In [ ]:
context_variables = sk.ContextVariables(variables={"min": "1", "max": "5", "language": "Spanish"})
num = await generate_number.invoke(variables=context_variables)
context_variables["paragraph_count"] = str(num.result)

In [ ]:
# Pass the output to the semantic story function
story = await corgi_story.invoke(variables=context_variables)

In [ ]:
print(
    "Generating a corgi story exactly {} paragraphs long in {} language: ".format(
        context_variables["paragraph_count"], context_variables["language"]
    )
)
print("=====================================================")
print(story)

### Recap

A quick review of what we've learned here:

- We've learned how to create native and semantic functions and register them to the kernel
- We've seen how we can use context variables to pass in more custom variables into our prompt
- We've seen how we can call native functions within semantic function prompts.
